In [12]:
# Make sure libraries are reloaded when this cell is run
%load_ext autoreload
%autoreload 2

from astropy import units as u

from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import OrbitPlotter3D
import copy
import CustomManeuvres
from poliastro.twobody.thrust.change_a_inc import change_a_inc
from poliastro._math.linalg import norm
import poliastro.core.angles as angles
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
# Define the two coplanar debris
a = 6000+6371 << u.km
ecc = 0 << u.one
inc = 0 << u.deg
raan = 0 << u.deg
argp = 0 << u.deg
nu = 60 << u.deg

deb_1 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

a = 3000+6371 << u.km
inc = 0 << u.deg
nu = 0 << u.deg

deb_2 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

# Init the otv at debris 1
otv = copy.copy(deb_1)

In [81]:
print(deb_1.epoch.iso)
print(deb_2.epoch.iso)
print(otv.epoch.iso)

2000-01-01 12:00:00.000
2000-01-01 12:00:00.000
2000-01-01 12:00:00.000


In [20]:
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

In [8]:
import CustomManeuvres
inc_change = CustomManeuvres.simple_inc_change(otv, deb_2)

# Get the transfer time of the hoh_phas
transfer_time = inc_change.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)

# Apply the maneuver to the otv
otv = otv.apply_maneuver(inc_change)

print('otv inc', otv.inc << u.deg)
print('deb_2 inc', deb_2.inc)

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

currrent_anomaly 60.0 deg
thrust_location 179.999 deg
time_to_thrust 4564.501109595941 s
thrust_norm 0.0 m / s
y_thrust 0.0 m / s
z_thrust -0.0 m / s
otv inc 0.0 deg
deb_2 inc 0.0 deg


In [21]:
print(deb_1.nu << u.deg)
print(otv.nu << u.deg)
print(deb_2.nu << u.deg)
print((otv.nu + otv.argp) << u.deg)

# mean_anomaly = angles.nu_to_M(otv.nu, otv.ecc)

# print(mean_anomaly << u.deg)

60.0 deg
60.0 deg
0.0 deg
60.0 deg


In [23]:
# Calculate a basic homnann
import CustomManeuvres
hoh_phas = CustomManeuvres.hohmann_with_phasing(otv, deb_2)
hoh_phas

Target Delta:  315.09565701645073 deg
current_delta :  300.0 deg
t_1:  25385.860920295116 s
mean_anomaly_i:  7.384638853589243 deg
new delta:  -75.09565701645064 deg


Number of impulses: 2, Total cost: 0.841563 km / s

In [18]:
hoh_phas.impulses

((<Quantity 33931.51750328 s>,
  <Quantity [297.99784075,  74.61079575,   0.        ] m / s>),
 (<Quantity 5255.47453664 s>,
  <Quantity [-283.91309118,  -71.08434613,    0.        ] m / s>))

In [17]:
# Get the transfer time of the hoh_phas
transfer_time = hoh_phas.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)

# Propagate the otv using the calculated thrust sequence
transfer_orbit, otv = otv.apply_maneuver(hoh_phas, intermediate=True)

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)
frame.plot(transfer_orbit)



In [18]:
# Wrong values, need to use mean anomaly to compare
print("OTV ANGLE: ", otv.nu.to(u.deg))
print("DEB2 ANGLE: ", deb_2.nu.to(u.deg))
print('difference: ', (otv.nu - deb_2.nu).to(u.deg))
print('OTV.a: ', otv.a)
print('Deb2.a: ', deb_2.a)

OTV ANGLE:  -172.61536114641075 deg
DEB2 ANGLE:  157.19332482068768 deg
difference:  -329.8086859670984 deg
OTV.a:  9371.0 km
Deb2.a:  9371.0 km
